# Recommender - Categorical Cross Entropy

Recommender system with categorical cross entropy loss function with cross validation

In [28]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [29]:
from helpers import load_data

DATA_TRAIN_PATH = "data/data_train.csv"
ratings = load_data(DATA_TRAIN_PATH)

DATA_TEST_PATH = "data/sampleSubmission.csv"
samples = load_data(DATA_TEST_PATH)

number of items: 10000, number of users: 1000
number of items: 10000, number of users: 1000


## Data - preprocessing

In [81]:
X = data[['user_id', 'movie_id']].values
Y = data[['rating']].values

In [82]:
from sklearn.model_selection import train_test_split, KFold
# split of the data: 0.9 train, 0.1 test
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.1, random_state=42)

encoder = LabelEncoder()
encoder.fit(Y_train)
encoder_rate = encoder.transform(Y_train)

Y_train_dummy= np_utils.to_categorical(encoder_rate)

## Neural network model 

In [83]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(10, input_dim=2, activation='relu'))
    model.add(Dense(5, activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [84]:
n_split = 10
for train_index, test_index in KFold(n_split).split(X_train):
    x_train,x_test = X_train[train_index], X_train[test_index]
    y_train,y_test = Y_train_dummy[train_index], Y_train_dummy[test_index]
    
    model = baseline_model()
    history = model.fit(x=x_train, y=y_train,  batch_size=10000, epochs=10,verbose=1, validation_data=(x_test, y_test))

Train on 953330 samples, validate on 105926 samples
Epoch 1/10
953330/953330 [==============================] - 2s 2us/step - loss: 6.5752 - accuracy: 0.1420 - val_loss: 5.8302 - val_accuracy: 0.0866
Epoch 2/10
953330/953330 [==============================] - 1s 1us/step - loss: 5.7293 - accuracy: 0.0853 - val_loss: 5.7050 - val_accuracy: 0.0848
Epoch 3/10
953330/953330 [==============================] - 1s 1us/step - loss: 4.1261 - accuracy: 0.0807 - val_loss: 1.6484 - val_accuracy: 0.0479
Epoch 4/10
953330/953330 [==============================] - 1s 1us/step - loss: 1.6122 - accuracy: 0.0395 - val_loss: 1.6095 - val_accuracy: 0.0388
Epoch 5/10
953330/953330 [==============================] - 1s 1us/step - loss: 1.6095 - accuracy: 0.0382 - val_loss: 1.6095 - val_accuracy: 0.0389
Epoch 6/10
953330/953330 [==============================] - 1s 1us/step - loss: 1.6095 - accuracy: 0.0383 - val_loss: 1.6094 - val_accuracy: 0.0390
Epoch 7/10
953330/953330 [==============================] - 

In [88]:
from sklearn.metrics import mean_squared_error


prediction_test = model.predict(x=X_test)



#print (mean_squared_error(Y_test, prediction_test))

NameError: name 'prdiction_test' is not defined

In [1]:
prediction_test[:3]

NameError: name 'prediction_test' is not defined